In [9]:
import ydb
import json
from datetime import datetime, timezone

def connect_db():
    IAM = 't1.9euelZqWk5DJypqNyZmRyJqJms6emO3rnpWazcrOjo3Nnc3LjpSMkcnJxsjl9PczHVpG-e9vLDfX3fT3c0tXRvnvbyw3183n9euelZqUls_Oi5mLyM-Qi5yKzI-RjO_8xeuelZqUls_Oi5mLyM-Qi5yKzI-RjA._Z4KsjK7PzSiPVOSuExjYOgQ-T5rcFfx2jFbSpmltt6DTnyfffksU4483MtVORBW7tKSqpB5m9sFea09NG-nDw'  # Заменить на действительный IAM токен
    driver_config = ydb.DriverConfig(
        endpoint='grpcs://ydb.serverless.yandexcloud.net:2135',
        database='/ru-central1/b1gsde16rgc0mhkhj1t4/etnjgop9305nqltdo5ob',
        credentials=ydb.credentials.AccessTokenCredentials(f'Bearer {IAM}')
    )
    driver = ydb.Driver(driver_config)
    driver.wait(timeout=5)
    return driver

def get_old_timestamp():
    return datetime(1970, 1, 1, tzinfo=timezone.utc)

def get_current_timestamp():
    return datetime.now(timezone.utc)

def check_user_exists(driver, chat_id):
    session = driver.table_client.session().create()
    query = f"""
    SELECT shikimori_username
    FROM users
    WHERE chat_id = {chat_id}
    """
    result_set = session.transaction(ydb.SerializableReadWrite()).execute(query, commit_tx=True)
    return result_set[0].rows if result_set else None

def add_user(driver, chat_id):
    session = driver.table_client.session().create()
    last_updated = get_old_timestamp().strftime('%Y-%m-%dT%H:%M:%SZ')
    query = f"""
    INSERT INTO users (chat_id, shikimori_username, last_updated)
    VALUES ({chat_id}, NULL, CAST('{last_updated}' AS TIMESTAMP))
    """
    session.transaction(ydb.SerializableReadWrite()).execute(query, commit_tx=True)

def update_last_updated(driver, chat_id):
    session = driver.table_client.session().create()
    last_updated = get_current_timestamp().strftime('%Y-%m-%dT%H:%M:%SZ')
    query = f"""
    UPDATE users
    SET last_updated = CAST('{last_updated}' AS TIMESTAMP)
    WHERE chat_id = {chat_id}
    """
    session.transaction(ydb.SerializableReadWrite()).execute(query, commit_tx=True)

def add_shiki_to_user(driver, chat_id, shiki_username):
    session = driver.table_client.session().create()
    query = f"""
    UPDATE users
    SET shikimori_username = '{shiki_username}'
    WHERE chat_id = {chat_id}
    """
    session.transaction(ydb.SerializableReadWrite()).execute(
        query,
        commit_tx=True
    )

def get_context_data(driver, chat_id):
    session = driver.table_client.session().create()

    query_median_year = f"""
    SELECT PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY a.anime_id) AS median_year
    FROM user_anime ua
    JOIN anime a ON ua.anime_id = a.anime_id
    WHERE ua.chat_id = {chat_id} AND ua.status = 'finished'
    """

    query_top_genre = f"""
    SELECT genre
    FROM (
        SELECT json_value(uad.details, '$.genre') AS genre, COUNT(*) AS count
        FROM user_anime ua
        LEFT JOIN anime a ON ua.anime_id = a.anime_id
        LEFT JOIN unnest(json_query_array(a.geners, '$')) AS uad(details) ON true
        WHERE ua.chat_id = {chat_id} AND ua.status = 'finished'
        GROUP BY genre
        ORDER BY count DESC
        LIMIT 1
    ) t
    """

    query_median_episodes = f"""
    SELECT PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY a.episode_count) AS median_episodes
    FROM user_anime ua
    JOIN anime a ON ua.anime_id = a.anime_id
    WHERE ua.chat_id = {chat_id} AND ua.status = 'finished'
    """

    query_top_bottom_titles = f"""
    (SELECT array_agg(a.title ORDER BY ua.user_score DESC LIMIT 3) AS top_titles
    FROM user_anime ua
    JOIN anime a ON ua.anime_id = a.anime_id
    WHERE ua.chat_id = {chat_id} AND ua.status = 'finished'),
    (SELECT array_agg(a.title ORDER BY ua.user_score ASC LIMIT 3) AS bottom_titles
    FROM user_anime ua
    JOIN anime a ON ua.anime_id = a.anime_id
    WHERE ua.chat_id = {chat_id} AND ua.status = 'finished')
    """

    # Выполнение запросов
    median_year_result = session.transaction(ydb.SerializableReadWrite()).execute(query_median_year)
    top_genre_result = session.transaction(ydb.SerializableReadWrite()).execute(query_top_genre)
    median_episodes_result = session.transaction(ydb.SerializableReadWrite()).execute(query_median_episodes)
    top_bottom_titles_result = session.transaction(ydb.SerializableReadWrite()).execute(query_top_bottom_titles)

    context_data = {
        'median_year': median_year_result[0].rows[0].median_year,
        'top_genre': top_genre_result[0].rows[0].genre,
        'median_episodes': median_episodes_result[0].rows[0].median_episodes,
        'top_titles': top_bottom_titles_result[0].rows[0].top_titles,
        'bottom_titles': top_bottom_titles_result[0].rows[0].bottom_titles
    }

    return context_data

def load_user_anime_list(driver, chat_id, user_anime_list):
    session = driver.table_client.session().create()

    for anime in user_anime_list:
        anime_id = anime['anime_id']
        user_anime_id_query = f"""
        SELECT user_anime_id FROM user_anime WHERE chat_id = {chat_id} AND anime_id = {anime_id}
        """
        user_anime_result = session.transaction(ydb.SerializableReadWrite()).execute(user_anime_id_query)

        if not user_anime_result[0].rows:
            user_anime_insert_query = f"""
            INSERT INTO user_anime (user_anime_id, chat_id, anime_id, status, user_score)
            VALUES ('{generate_id()}', {chat_id}, {anime_id}, '{anime['status']}', {anime['score']})
            """
            session.transaction(ydb.SerializableReadWrite()).execute(user_anime_insert_query, commit_tx=True)

        anime_record_query = f"SELECT anime_id FROM anime WHERE anime_id = {anime_id}"
        anime_record_result = session.transaction(ydb.SerializableReadWrite()).execute(anime_record_query)

        if not anime_record_result[0].rows:
            add_anime(driver, anime)

def add_anime(session, anime_details):
    insert_anime_query = f"""
    INSERT INTO anime (anime_id, title, average_score, geners, type, episode_duration, episode_count, poster_url, description)
    VALUES ({anime_details['anime_id']}, '{anime_details['name']}', {anime_details['average_score']},
    '{json.dumps(anime_details['genres'])}', '{anime_details['type']}', {anime_details['episode_duration']},
    {anime_details['episode_count']}, '{anime_details['poster_url']}', '{anime_details['description']}')
    """
    session.transaction(ydb.SerializableReadWrite()).execute(insert_anime_query, commit_tx=True)

def generate_id():
    import uuid
    return str(uuid.uuid4())

In [10]:
driver_test = connect_db()

In [13]:
add_user(driver_test, 12345)

In [16]:
add_shiki_to_user(driver_test, 12345, 11121)

In [12]:
update_last_updated(driver_test, 231231)

In [ ]:
get_context_data(driver_test, 231231)

In [12]:
check_user_exists(driver_test, 505096036)

[{'shikimori_username': None}]

In [23]:
user_info = check_user_exists(driver_test, 12345)

In [24]:
user_info

[{'shikimori_username': b'11121'}]

In [25]:
user_info[0].get('shikimori_username')

b'11121'

In [37]:
import requests
import os
import json

BOT_TOKEN = '8046820225:AAHsGMS7vJIg-CbrqfV3MuOK5_N_8zKwI-U'
BASE_URL = f"https://api.telegram.org/bot{BOT_TOKEN}"

def send_message(chat_id, text, photo_url=None, command_buttons=None, inline_button_text=None, inline_button_url=None):
    # Подготовка клавиатуры для команд, если переданы соответствующие параметры
    reply_markup = None

    if command_buttons:
        keyboard = [[{'text': cmd}] for cmd in command_buttons]
        markup = {"keyboard": keyboard, "resize_keyboard": True, "one_time_keyboard": True}
    else:
        markup = {}

    # Подготовка встроенной клавиатуры, если переданы параметры для ссылки
    if inline_button_text and inline_button_url:
        inline_markup = {
            "inline_keyboard": [[{"text": inline_button_text, "url": inline_button_url}]]
        }
        reply_markup = inline_markup
    else:
        reply_markup = markup if command_buttons else None

    # Подготовка данных и файлов для запроса
    if photo_url:
        url = f"{BASE_URL}/sendPhoto"
        data = {'chat_id': chat_id, 'caption': text, 'reply_markup': json.dumps(reply_markup) if reply_markup else None}
        files = {'photo': requests.get(photo_url).content}
    else:
        url = f"{BASE_URL}/sendMessage"
        data = {'chat_id': chat_id, 'text': text, 'reply_markup': json.dumps(reply_markup) if reply_markup else None}
        files = None

    try:
        response = requests.post(url, data=data, files=files)
        response.raise_for_status()  # Проверка наличия HTTP ошибок
    except requests.RequestException as e:
        print(f"Error sending message: {e}")

    return response.json()

In [38]:
send_message(
    505096036,
    text="Похоже, вы ещё не привязали аккаунт Shikimori. Это поможет учесть ваши оценки и список при рекомендациях.",
    command_buttons=["/add_shiki"]
)

{'ok': True,
 'result': {'message_id': 11,
  'from': {'id': 8046820225,
   'is_bot': True,
   'first_name': 'Shiki',
   'username': 'Skikinormbot'},
  'chat': {'id': 505096036,
   'first_name': 'Пётр',
   'last_name': 'Мирошин',
   'username': 'King2238',
   'type': 'private'},
  'date': 1730767426,
  'text': 'Похоже, вы ещё не привязали аккаунт Shikimori. Это поможет учесть ваши оценки и список при рекомендациях.'}}

In [29]:
import requests
import os
import json

BOT_TOKEN = '8046820225:AAHsGMS7vJIg-CbrqfV3MuOK5_N_8zKwI-U'
def send_message(chat_id, text, photo_url=None, button_text=None, button_url=None):
    base_url = f"https://api.telegram.org/bot{BOT_TOKEN}/"

    # Подготовка кнопки, если предоставлены параметры
    reply_markup = None
    if button_text and button_url:
        reply_markup = {
            "inline_keyboard": [[{"text": button_text, "url": button_url}]]
        }

    # Отправка сообщения с картинкой или без
    if photo_url:
        url = base_url + "sendPhoto"
        data = {
            'chat_id': chat_id,
            'caption': text,
            'reply_markup': json.dumps(reply_markup) if reply_markup else None
        }
        files = {'photo': requests.get(photo_url).content}
    else:
        url = base_url + "sendMessage"
        data = {
            'chat_id': chat_id,
            'text': text,
            'reply_markup': json.dumps(reply_markup) if reply_markup else None
        }
        files = None

    try:
        response = requests.post(url, data=data, files=files)
        response.raise_for_status()  # Проверка на наличие HTTP ошибок
    except requests.RequestException as e:
        print(f"Error sending message: {e}")
    return response.json()

In [31]:
send_message(505096036, 
             text="Добро пожаловать! Я бот, который поможет вам с рекомендованными аниме.\nИспользуйте команды для получения рекомендаций.",
             button_text="Получить рекомендации",
             button_url="/get_recd")

Error sending message: 400 Client Error: Bad Request for url: https://api.telegram.org/bot8046820225:AAHsGMS7vJIg-CbrqfV3MuOK5_N_8zKwI-U/sendMessage


{'ok': False,
 'error_code': 400,
 'description': "Bad Request: inline keyboard button URL '/get_recd' is invalid: URL host is empty"}